In [2]:
import msgpack
import numpy

In [150]:
arr = numpy.random.randint(0, 255, (3, 1280, 720, 3), dtype=numpy.uint8)

In [ ]:
# %%timeit -n 1

import einops
import imageio

image_format = "jpeg"
image_encoded = arr
image_encoded_shape = einops.parse_shape(image_encoded, "b h w c")
image_encoded = einops.rearrange(image_encoded, "b h w c -> (b h) w c")
image_encoded = imageio.imwrite(
    "<bytes>",
    image_encoded,
    format=image_format,
)

image_decoded = imageio.imread(
    image_encoded,
    format=image_format,
)
image_decoded = einops.rearrange(image_decoded, "(b h) w c -> b h w c", **image_encoded_shape)
image_decoded.shape

26.6 ms ± 851 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


<magic-timeit>:16: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


In [183]:
len(arr.tobytes()) / 1e6

8.2944

In [185]:
len(image_encoded) / 1e6

1.659199

In [149]:
numpy.allclose(arr, image_decoded)

True

In [ ]:
import io

import msgpack
import numpy


def _encode_tensor(arr: ...):
    buffer = io.BytesIO()
    numpy.save(
        buffer,
        arr=arr,
        allow_pickle=False,
    )
    return buffer.getvalue()


def _decode_tensor(bytes: ...):
    return numpy.load(bytes, allow_pickle=False)


from xvla_wlr.model import Action, Observation

def _encode_observation(observation: Observation):
    return msgpack.dumps({
        "text": _encode_tensor(observation.text),
        "images": _encode_tensor(observation.images),
        "images_mask": _encode_tensor(observation.images_mask),
        "domain_id": _encode_tensor(observation.domain_id),
        "ee_transform": _encode_tensor(observation.ee_transform),
        "ee_gripper_val": _encode_tensor(observation.ee_gripper_val),
    })

    
    ...

%timeit -n 10 _ = _encode_observation(Observation.sample())

497 μs ± 28 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [225]:
%%timeit -n 10
import io
import numpy

buffer = io.BytesIO()
arr = numpy.empty((3, 224, 224, 3))
numpy.save(
    buffer,
    arr=arr,
    allow_pickle=False,
)
buffer.getvalue()

The slowest run took 4.96 times longer than the fastest. This could mean that an intermediate result is being cached.
434 μs ± 302 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
numpy.load

In [205]:
len(buffer.getvalue()) / 1e6

3.6128

In [41]:
%timeit -n100 arr.tobytes()

76.9 μs ± 33.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [219]:
%%timeit -n100 
buffer = io.BytesIO()
numpy.save(buffer, arr=numpy.full((3, 20), "yee"), allow_pickle=False)

5.59 μs ± 385 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%timeit -n10 arr.tolist()

9.56 ms ± 756 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
from typing import Literal, TypedDict


class Message(TypedDict):
    observation: ...



b"\x93NUMPY\x01\x00v\x00{'descr': '<f8', 'fortran_order': False, 'shape': (10,), }                                                           \n\xaaH\xcf$\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00f4ceba03-3df6-45f6-932b-df2161cd3832\xa3~\x00\x00"